In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import numpy as np
import os,sys
from PIL import Image

from unet import *
from augmentation import *


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Using TensorFlow backend.


## We start by initializing a convolutional neural network based on the U-net architecture. 

The summary shows how the layers are structured with input-shape, functions and number of neurons. You can see that the input has the same shape as the output.

In [2]:
#create the cnn
model = get_unet()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxP

/content/unet.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


## We now declare our variables for training the network


In [0]:
#variables for how random images will be generated when training
generator_variables = dict(rotation_range=90,
                           width_shift_range=0.0,
                           height_shift_range=0.0,
                           zoom_range=0.7,
                           horizontal_flip=True,
                           vertical_flip = True,
                           fill_mode='reflect')
#number of images per gradient-calculation
batch_size = 2
#number of times we try gradients per epoch
steps_per_epoch = 600
epochs= 10

## Specify where the training images are stored, and where the model should be saved

In [0]:

#fetch images from drive
cloud_dir = "drive/My Drive/"
train_dir = cloud_dir+"training/"
#directory where checkpoint weights will be stored
weights_dir = train_dir+"weights/"
weights = weights_dir+'weights.hdf5'
img_folder = 'images'
gtruth_folder = 'gt'


## Train the model using the generator-variables we previously specified
By using checkpoint the model will be saved in the specified folder

In [0]:
#we make a generator that will be used for training
train_generator = trainGenerator(batch_size,train_dir,img_folder,gtruth_folder,generator_variables)
#we save the weights to a file
checkpoint = ModelCheckpoint(weights, monitor='loss', verbose=1, save_best_only=True)
#we train the model using the generator
model.fit_generator(train_generator,steps_per_epoch=steps_per_epoch,epochs=epochs, callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.
161/600 [=======>......................] - ETA: 5:21 - loss: 0.1782 - acc: 0.9681

## We now have a trained model that can be used to perform image segmentation of craters

